In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [4]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [5]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.math.subtract(y_pred, 42)
    b = dist

    loss = tf.math.add(a, b)
    return loss

In [7]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(16, activation='relu'))
generator.add(tf.keras.layers.Dense(8, activation='relu'))
generator.add(tf.keras.layers.Dense(4, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [8]:
def return_latest():
    df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    latest = X[-1].reshape(1, 21)

    return tf.cast(latest, dtype=tf.float32)

In [9]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    real_data = return_latest()
    loss = my_loss(real_data, X_pred)
    generator.train_on_batch(X_pred, real_data)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[0.732772]], pop: [[40.941864]]
epoch: 1, loss: [[0.6740222]], pop: [[40.885857]]
epoch: 2, loss: [[0.68078804]], pop: [[40.888863]]
epoch: 3, loss: [[0.4281428]], pop: [[40.633125]]
epoch: 4, loss: [[0.59025383]], pop: [[40.788525]]
epoch: 5, loss: [[0.11148953]], pop: [[40.30367]]
epoch: 6, loss: [[0.30799437]], pop: [[40.52205]]
epoch: 7, loss: [[0.18405247]], pop: [[40.374928]]
epoch: 8, loss: [[0.00848114]], pop: [[40.21954]]
epoch: 9, loss: [[0.38425875]], pop: [[40.604576]]
epoch: 10, loss: [[-0.2449807]], pop: [[39.944695]]
epoch: 11, loss: [[0.30374467]], pop: [[40.52893]]
epoch: 12, loss: [[-0.9393399]], pop: [[39.211617]]
epoch: 13, loss: [[-0.5828061]], pop: [[39.59522]]
epoch: 14, loss: [[-0.12725055]], pop: [[40.087955]]
epoch: 15, loss: [[-0.15267026]], pop: [[40.06327]]
epoch: 16, loss: [[-0.04037499]], pop: [[40.182095]]
epoch: 17, loss: [[-1.1165121]], pop: [[39.062477]]
epoch: 18, loss: [[-1.4628123]], pop: [[38.692707]]
epoch: 19, loss: [[-0.5315844

In [10]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [14]:
print(pop_result.max())
print(pop_result.min())
print(pop_result.mean())

40.941864
25.668583
28.51577


In [11]:
df = np.append(X_result, pop_result, axis=1)

In [12]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/basic_formula.xlsx')